In [1]:
import pandas as pd
import numpy as np
from numpy import array
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import optimizers

2025-10-14 16:38:08.398046: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("PRSA_Data_20130301-20170228/PRSA_Data_Aotizhongxin_20130301-20170228.csv")

In [3]:
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,95.0,12.5,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Aotizhongxin


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   No       35064 non-null  int64  
 1   year     35064 non-null  int64  
 2   month    35064 non-null  int64  
 3   day      35064 non-null  int64  
 4   hour     35064 non-null  int64  
 5   PM2.5    34139 non-null  float64
 6   PM10     34346 non-null  float64
 7   SO2      34129 non-null  float64
 8   NO2      34041 non-null  float64
 9   CO       33288 non-null  float64
 10  O3       33345 non-null  float64
 11  TEMP     35044 non-null  float64
 12  PRES     35044 non-null  float64
 13  DEWP     35044 non-null  float64
 14  RAIN     35044 non-null  float64
 15  wd       34983 non-null  object 
 16  WSPM     35050 non-null  float64
 17  station  35064 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 4.8+ MB


In [5]:
df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,34139.000000,34346.000000,34129.000000,34041.000000,33288.000000,33345.000000,35044.000000,35044.000000,35044.000000,35044.000000,35050.000000
mean,17532.500000,2014.662560,6.522930,15.729637,11.500000,82.773611,110.060391,17.375901,59.305833,1262.945145,56.353358,13.584607,1011.846920,3.123062,0.067421,1.708496
std,10122.249256,1.177213,3.448752,8.800218,6.922285,82.135694,95.223005,22.823017,37.116200,1221.436236,57.916327,11.399097,10.404047,13.688896,0.910056,1.204071
min,1.000000,2013.000000,1.000000,1.000000,0.000000,3.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-16.800000,985.900000,-35.300000,0.000000,0.000000
25%,8766.750000,2014.000000,4.000000,8.000000,5.750000,22.000000,38.000000,3.000000,30.000000,500.000000,8.000000,3.100000,1003.300000,-8.100000,0.000000,0.900000
50%,17532.500000,2015.000000,7.000000,16.000000,11.500000,58.000000,87.000000,9.000000,53.000000,900.000000,42.000000,14.500000,1011.400000,3.800000,0.000000,1.400000
75%,26298.250000,2016.000000,10.000000,23.000000,17.250000,114.000000,155.000000,21.000000,82.000000,1500.000000,82.000000,23.300000,1020.100000,15.600000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,898.000000,984.000000,341.000000,290.000000,10000.000000,423.000000,40.500000,1042.000000,28.500000,72.500000,11.200000


In [6]:
df.isnull().sum()

No            0
year          0
month         0
day           0
hour          0
PM2.5       925
PM10        718
SO2         935
NO2        1023
CO         1776
O3         1719
TEMP         20
PRES         20
DEWP         20
RAIN         20
wd           81
WSPM         14
station       0
dtype: int64

In [7]:
df = df.dropna()

In [8]:
df.isnull().sum()

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

In [9]:
df.shape

(31815, 18)

In [10]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


#### split into train and test

In [11]:
train_size = int(df.shape[0]*0.8)
test_size = df.shape[0] - train_size

- There are many pollutants. Let's first try to predict PM2.5 concentration values.
- Let the years 2016 and 2017 be the testing set.
- As you can see below, these 2 years account for 20.06% of the data (test set).

In [12]:
df.loc[train_size:].count() / df.shape[0] * 100

No         28.310545
year       28.310545
month      28.310545
day        28.310545
hour       28.310545
PM2.5      28.310545
PM10       28.310545
SO2        28.310545
NO2        28.310545
CO         28.310545
O3         28.310545
TEMP       28.310545
PRES       28.310545
DEWP       28.310545
RAIN       28.310545
wd         28.310545
WSPM       28.310545
station    28.310545
dtype: float64

In [13]:
train_size, test_size

(25452, 6363)

In [14]:
X_train = df[:train_size]
X_test = df[train_size:]

y_train = X_train["PM2.5"]
y_test = X_test["PM2.5"]

In [15]:
X_train

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28277,28278,2016,5,22,5,62.0,86.0,9.0,92.0,800.0,2.0,17.2,1015.5,8.9,0.0,NE,0.9,Aotizhongxin
28278,28279,2016,5,22,6,56.0,71.0,8.0,86.0,700.0,6.0,18.2,1015.5,8.5,0.0,ENE,1.1,Aotizhongxin
28279,28280,2016,5,22,7,54.0,73.0,9.0,85.0,700.0,8.0,19.3,1015.6,7.7,0.0,ENE,1.1,Aotizhongxin
28280,28281,2016,5,22,8,46.0,61.0,9.0,61.0,700.0,52.0,21.4,1015.8,7.6,0.0,N,1.9,Aotizhongxin


In [16]:
y_train

0         4.0
1         8.0
2         7.0
3         6.0
4         3.0
         ... 
28277    62.0
28278    56.0
28279    54.0
28280    46.0
28281    54.0
Name: PM2.5, Length: 25452, dtype: float64

In [17]:
X_test

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
28282,28283,2016,5,22,10,37.0,83.0,7.0,21.0,600.0,126.0,25.5,1015.2,8.0,0.0,N,1.9,Aotizhongxin
28283,28284,2016,5,22,11,37.0,78.0,7.0,20.0,600.0,146.0,27.0,1014.4,7.9,0.0,NNE,1.4,Aotizhongxin
28284,28285,2016,5,22,12,38.0,71.0,7.0,23.0,600.0,164.0,27.7,1013.6,8.0,0.0,S,1.2,Aotizhongxin
28285,28286,2016,5,22,13,49.0,69.0,7.0,20.0,500.0,200.0,28.7,1012.8,4.9,0.0,SE,2.1,Aotizhongxin
28286,28287,2016,5,22,14,41.0,57.0,8.0,14.0,500.0,223.0,29.1,1011.6,5.8,0.0,S,2.1,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,95.0,12.5,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Aotizhongxin


In [18]:
y_test

28282    37.0
28283    37.0
28284    38.0
28285    49.0
28286    41.0
         ... 
35059    12.0
35060    13.0
35061    16.0
35062    21.0
35063    19.0
Name: PM2.5, Length: 6363, dtype: float64

#### Normalize the data

In [19]:
train_norm = X_train["PM2.5"]
train_norm_arr = np.asarray(train_norm)
train_norm = np.reshape(train_norm_arr, (-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
train_norm = scaler.fit_transform(train_norm)

- Even after normalization and scaing, null values are possible (many people disregard this). Let's check if any null values are present.

In [20]:
count = 0

for i in range(len(train_norm)):
    if train_norm[i] == 0:
        count += 1

print("Number of null values in train_norm = ", count)

Number of null values in train_norm =  301


In [21]:
train_norm = train_norm[train_norm != 0]

In [22]:
test_norm = X_test["PM2.5"]
test_norm_arr = np.asarray(test_norm)
test_norm = np.reshape(test_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
test_norm = scaler.fit_transform(test_norm)

In [23]:
count = 0

for i in range(len(test_norm)):
    if test_norm[i] == 0:
        count += 1

print("Number of null values in test_norm = ", count)

Number of null values in test_norm =  124


In [24]:
test_norm = test_norm[test_norm != 0]

#### Split data using timestamp

- Since this is a time seris data, we should be predicting the values after looking at a set of values rather than just a single value like we usually do.
- This takes into account the correlation between the data points and the timestamps.
- Because the neighbours should be considered for how the values change over time.

Let's define a function to do this.

The below function called split_sequence splits the sequence into sets of n values. This n is given as n_steps (step_size).

For example,
if n=3, we split the sequence in groups of 3. We create 2 empty lists and append the split sequences.


In [25]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()

    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence

        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

In [26]:
n_steps = 3
n_features = 1

In [27]:
# Train split
X_split_train, y_split_train = split_sequence(train_norm, n_steps)

# Test split
X_split_test, y_split_test = split_sequence(test_norm, n_steps)

In [28]:
X_split_train = X_split_train.reshape(
    (X_split_train.shape[0], X_split_train.shape[1], n_features))

X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

In [29]:
X_split_train

array([[[0.00140845],
        [0.00704225],
        [0.0056338 ]],

       [[0.00704225],
        [0.0056338 ],
        [0.00422535]],

       [[0.0056338 ],
        [0.00422535],
        [0.0028169 ]],

       ...,

       [[0.08873239],
        [0.08309859],
        [0.07464789]],

       [[0.08309859],
        [0.07464789],
        [0.07183099]],

       [[0.07464789],
        [0.07183099],
        [0.06056338]]])

In [30]:
for i in range(5):
    print(X_split_test[i], y_split_test[i])

[[0.04788732]
 [0.04788732]
 [0.04929577]] 0.06478873239436621
[[0.04788732]
 [0.04929577]
 [0.06478873]] 0.053521126760563385
[[0.04929577]
 [0.06478873]
 [0.05352113]] 0.060563380281690136
[[0.06478873]
 [0.05352113]
 [0.06056338]] 0.06338028169014086
[[0.05352113]
 [0.06056338]
 [0.06338028]] 0.059154929577464786


#### Create a LSTM Model

- Let's define our neural network LSTM(Long-Short Term Memory).
- Let's add 50 nodes in our first layer with a ReLU(Rectifier Linear Unit) as a activation function.
- Their shape will be step size, number of features.
- Then we will add a dense layer with one node for the output.
- We can try out different optimizers to see which minimizes loss and maximizes accuracy.
- Stochastic Gradient Descent(SGD), Adam, AdaBoost, RMSProp are few of them.
    - lr=learning rate,
    - decay=by how much to decay learning rate,
    - momentum=how much should the gradient descent be accelerated to dampen oscillations,
    - nesterov=whether to use nesterov momentum. Nesterov has stronger convergence for conver functions.
- We compile using MSE(Mean Squared Error) as our loss function.

In [31]:
model = Sequential()
model.add(LSTM(50, activation="relu", input_shape=(n_steps, n_features)))
model.add(Dense(1))

sgd = optimizers.SGD(learning_rate=0.01, weight_decay=1e-5, momentum=0.9, nesterov=True)

keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer='sgd', loss='mse', metrics=["accuracy"])

W0000 00:00:1760449119.747638  302538 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/ibrahim/anaconda3/envs/geo_clean/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [32]:
hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=13, verbose=1)

Epoch 1/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0098 - val_accuracy: 0.0000e+00 - val_loss: 0.0102
Epoch 2/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0081 - val_accuracy: 0.0000e+00 - val_loss: 0.0090
Epoch 3/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0072 - val_accuracy: 0.0000e+00 - val_loss: 0.0079
Epoch 4/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0062 - val_accuracy: 0.0000e+00 - val_loss: 0.0067
Epoch 5/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0053 - val_accuracy: 0.0000e+00 - val_loss: 0.0057
Epoch 6/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0046 - val_accuracy: 0.0000e+00 - val_loss: 0.0049
Epoch 7/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0039 - val_accuracy: 0.0000e+00 - val_loss: 0.0041
Epoch 8/13
786/786 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/s